In [1]:
#! METODO DESACOPLADO RAPIDO
import cmath as cm
import numpy as np

In [2]:
cos = np.cos
sin = np.sin

In [3]:
def calc_P(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1
    p = 0
    for m in range(len(V)):
        p += V[m]*(G[i][m]*(cos(O[i] - O[m])) + B[i][m]*(sin(O[i] - O[m])))

    return V[i]*p

def calc_Q(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1
    q = 0
    for m in range(len(V)):
        q += V[m]*(G[i][m]*(sin(O[i] - O[m])) - B[i][m]*(cos(O[i] - O[m])))

    return V[i]* q

def calc_Hkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    h = -B[i][i]*V[i]**2 - calc_Q(index, V, O, Y)
    return h

def calc_Nkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    n = (V[i]**-1)*(calc_P(index, V, O, Y) + G[i][i]*(V[i]**2))
    return n

def calc_Mkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    mm = -G[i][i]*(V[i]**2) + calc_P(index, V, O, Y)
    return mm

def calc_Lkk(index, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    i = index - 1 

    l = (V[i]**-1)*(calc_Q(index, V, O, Y) - B[i][i]*(V[i]**2))
    return l

def calc_Hkm(i, j, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    k = i - 1
    m = j - 1

    h = V[k]*V[m]*(G[k][m]*sin(O[k] - O[m]) - B[k][m]*cos(O[k] - O[m]))
    return h

def calc_Nkm(i, j, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    k = i - 1
    m = j - 1

    n = V[m]*(G[k][m]*cos(O[k] - O[m]) + B[k][m]*sin(O[k] - O[m]))
    return n

def calc_Mkm(i, j, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    k = i - 1
    m = j - 1

    mm = -V[k]*V[m]*(G[k][m]*cos(O[k] - O[m]) + B[k][m]*sin(O[k] - O[m]))
    return mm

def calc_Lkm(i, j, V, O, Y):
    G = np.real(Y)
    B = np.imag(Y)

    k = i - 1
    m = j - 1

    mm = V[k]*(G[k][m]*sin(O[k] - O[m]) - B[k][m]*cos(O[k] - O[m]))
    return mm



In [12]:
z12 = z21 = -(.01 + .01j)
z13 = z31 = -(.02 + .02j)
z23 = z32 = -(.03 + .03j)

Y = np.array([[ -(1/z12 + 1/z13), 1/z12, 1/z13], [1/z21, -(1/z12 + 1/z23), 1/z23] , [1/z31, 1/z32, -(1/z13 + 1/z32)]])

V =[1, 1, 1]
O = [0, 0, 0]

P = [-.4, -.3, 0]
Q = [.2, .3, 0]
print(Y)

[[ 75.        -75.j         -50.        +50.j
  -25.        +25.j        ]
 [-50.        +50.j          66.66666667-66.66666667j
  -16.66666667+16.66666667j]
 [-25.        +25.j         -16.66666667+16.66666667j
   41.66666667-41.66666667j]]


In [18]:
z12 = z21 = -(.01 + .01j)
z13 = z31 = -(.02 + .02j)
z23 = z32 = -(.03 + .03j)

Y = np.array([[ -(1/z12 + 1/z13), 1/z12, 1/z13], [1/z21, -(1/z12 + 1/z23), 1/z23] , [1/z31, 1/z32, -(1/z13 + 1/z32)]])

V =[1, 1, 1]
O = [0, 0, 0]

P = [-.4, -.3, 0]
Q = [.2, .3, 0]

for it in range(2):
    dps = []; dqs = []
    for b in (1, 2):
        Pcalc = calc_P(b, V, O, Y)
        Qcalc = calc_Q(b, V, O, Y)

        dp = P[b -1] - Pcalc
        dq = Q[b -1] - Qcalc
        dps.append(dp); dqs.append(dq)

    DP = np.array(dps)
    DQ = np.array(dqs)
    DPQ = np.block([DP, DQ]).T

    H = []; L = []; M = []; N = []
    for b in (1, 2):
        for v in (1, 2):
            if b == v:
                Hx = calc_Hkk(v, V, O, Y); Lx = calc_Lkk(v, V, O, Y); Mx = calc_Mkk(v, V, O, Y); Nx = calc_Nkk(v, V, O, Y)
            else:
                Hx = calc_Hkm(b, v, V, O, Y); Lx = calc_Lkm(b, v, V, O, Y); Mx = calc_Mkm(b, v, V, O, Y); Nx = calc_Nkm(b, v, V, O, Y)
            H.append(Hx); L.append(Lx); M.append(Mx); N.append(Nx)
    
    H = np.array(H).reshape((2, 2)); L = np.array(L).reshape((2, 2)); M = np.array(M).reshape((2, 2)); N = np.array(N).reshape((2, 2))
    J = np.block([[H, N], [M, L]])

    Jinv = np.linalg.inv(J)

    deltas = np.matmul(Jinv, DPQ)
    O[1 - 1] += deltas[0]
    V[1 - 1] += deltas[2]

    O[2 - 1] += deltas[1]
    V[2 - 1] += deltas[3]

    print(f'V = {V}   O = {O}')


V = [0.9973333333333333, 0.9979999999999999, 1]   O = [-0.014, -0.015000000000000001, 0]
V = [0.9972213782113636, 0.9978874206646642, 1]   O = [-0.014035459850207712, -0.015038323752716992, 0]


In [22]:
P3 = calc_P(3, V, O, Y)
Q3 = calc_Q(3, V, O, Y)

print(f'V = {V}')
print(f'O = {O}')
print(f'P3 = {P3}\nQ1= {Q3}')

V = [0.9972213782113636, 0.9978874206646642, 1]
O = [-0.014035459850207712, -0.015038323752716992, 0]
P3 = 0.7090111560103693
Q1= -0.49098851423094914
